# Virtual species

In this vignette, we provide a demonstration of how the different **SpeciesDistributionToolkit** functions can be chained together to rapidly create a virtual species, generate its range map, and sample points from it according to the predicted suitability.

In [380]:
using SpeciesDistributionToolkit
using CairoMakie
using Statistics
CairoMakie.activate!(px_per_unit = 6.0)

We start by defining the extent in which we want to create the virtual species. For the purpose of this example, we will use the country of Austria, as available in the GADM database:

In [ ]:
place = SpeciesDistributionToolkit.gadm("AUT")
extent = SpeciesDistributionToolkit.boundingbox(place)

In [ ]:
provider = RasterData(CHELSA2, BioClim)
L = SDMLayer{Float32}[SDMLayer(provider; layer=l, extent...) for l in ["BIO1", "BIO12"]]

In [ ]:
msk = mask!(L[1], place)
rescale!.([mask!(l, msk) for l in L])

In [ ]:
logistic(x, α, β) = 1 / (1 + exp((x-β)/α))
logistic(α, β) = (x) -> logistic(x, α, β)
f = [logistic(randn(), rand()) for _ in eachindex(L)]

In [ ]:
S = prod([f[i].(L[i]) for i in eachindex(L)])

In [ ]:
target_prevalence = 0.2
cutoff = quantile(S, 1-target_prevalence)

In [ ]:
thr = LinRange(extrema(S)..., 100)
prv = [count(nodata(S .>= t, false))/count(S) for t in thr]
cutoff = thr[findmin(abs.(prv .- target_prevalence))[2]]

In [ ]:
presencelayer = backgroundpoints((v -> v > cutoff ? v : 0.0).(S), 100)

In [ ]:
f = Figure(size=(1000, 500))
ax = Axis(f[1,1], aspect=DataAspect())
heatmap!(ax, S .> cutoff, colormap=["#cececebb", :green])
lines!(ax, place[1].geometry, color=:black)
scatter!(ax, presencelayer, color=:white, strokecolor=:black, strokewidth=2, markersize=10, label="Virtual presences")
tightlimits!(ax)
hidespines!(ax)
hidedecorations!(ax)
axislegend(ax, position=:lt, framevisible=false)
f